In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp
import os
from dotenv import load_dotenv

import pymysql
# 상위 폴더(../)에 있는 `.env` 파일 로드
dotenv_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".env"))
load_dotenv(dotenv_path)


True

In [12]:
## mysql 연결
os.getenv('user_id')
os.getenv('user_password')

mysql_url = f"jdbc:mysql://{os.getenv('host_ip')}:3306/{os.getenv('DATABASE')}?useSSL=false&allowPublicKeyRetrieval=true&useUnicode=true&characterEncoding=UTF-8"

In [17]:
# SparkSession 생성
spark = SparkSession.builder \
    .appName("HDFS to MySQL") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .getOrCreate()


25/02/03 17:35:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [23]:
# HDFS에 있는 모든 Parquet 파일 로드
df = spark.read.parquet("hdfs:///data/")
print(df.count())

136568


In [24]:
## 시간 형식 수정(yyyy.mm.dd HH:MM:SS)
convert_cols = ['comment_publish_date', 'publish_date']
for c in convert_cols:
    df = df.withColumn(c
                     , to_timestamp(col('comment_publish_date'), "yyyy-MM-dd'T'HH:mm:ss'Z'")
                     )

In [25]:
## 결측치 제거
df = df.dropna()

In [26]:
# PyMySQL로 MySQL 연결 (SQL 실행을 위해)
conn = pymysql.connect(
    host=host_ip,
    user=user_id,
    password=user_password,
    database=DATABASE,
)

cursor = conn.cursor()

# ✅ MySQL 테이블 `TRUNCATE`
truncate_query = "TRUNCATE TABLE tbCrawled_Youtube"
cursor.execute(truncate_query)

conn.commit()
cursor.close()
conn.close()


In [27]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/local/lib/mysql-connector-java-5.1.49-bin.jar pyspark-shell'
df.write.format('jdbc')\
    .options(
        url = mysql_url
        , driver = 'com.mysql.jdbc.Driver'
        , dbtable = 'tbCrawled_Youtube'
        , user = user_id
        , password = user_password
    )\
    .mode('append')\
    .save()

In [28]:
spark.stop()